In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

In [6]:
# Data reading
x_train = pd.read_csv('../../data/raw/x_train.csv', sep=',')
y_train = pd.read_csv('../../data/raw/y_train.csv', sep=',')
x_test = pd.read_csv('../../data/raw/x_test.csv', sep=',')

In [9]:
x_train_transit = x_train[['OSRM estimated distance','intervention on public roads','floor']]
y_train_transit = y_train[['delta departure-presentation']]
x_test_transit = x_test[['OSRM estimated distance','intervention on public roads','floor']]

# Create a predictive model for the 'delta departure-presentation'
# based on 'OSRM estimated distance', 'intervention on public roads' and 'floor'
polynomial_features= PolynomialFeatures(degree=3)
x_train_transit_poly = polynomial_features.fit_transform(x_train_transit)
model = LinearRegression()
model.fit(x_train_transit_poly, y_train_transit)

# Prediction of the 'delta selection-presentation'
x_test_transit_poly = polynomial_features.fit_transform(x_test_transit)
y_selection_presentation_predicted = y_train['delta selection-departure'].median() + model.predict(x_test_transit_poly)

# Create a submission file
submission = pd.concat([pd.DataFrame(x_test[['emergency vehicle selection']].values),
           pd.DataFrame(np.full((len(x_test), 1), y_train['delta selection-departure'].median())),
           pd.DataFrame(model.predict(x_test_transit_poly)),
           pd.DataFrame(y_selection_presentation_predicted)],
           axis=1)

submission.columns = list(y_train.columns.values)

submission.set_index('emergency vehicle selection', inplace=True)

# submission.to_csv('./submission.csv', sep=",")

#### Evaluation function ####
# from sklearn.metrics import r2_score
# 
# r2_score(observed_values,predited_values)
#############################